In [ ]:
#prepare files for TFBS gat enrichment test

In [1]:
import pandas as pd
from pybedtools import BedTool
import os
import io
#import argparse

In [2]:
# parser = argparse.ArgumentParser(description='prepare_TFBS_enrichment')
# parser.add_argument('file_names', type=str, help='Name of folder and filenames for the promoters extracted')
# parser.add_argument('promoterpref', type=str, help='Promoter prefix name')
# parser.add_argument('Czechowski_gene_categories', type=str, help='Input location of Czechowski gene categories text file')
# parser.add_argument('promoter_bed_file', type=str, help='Input location of promoters bed file')
# parser.add_argument('output_genecat_prefix', type=str, help='Gene category prefix (eg. Czechowski)')
# parser.add_argument('mapped_motifs', type=str, help='input location of mapped_motifs bed file')#
# parser.add_argument('mapped_motifs_out', type=str, help='output location of mapped_motifs bed file with TF family in column 4')
# args = parser.parse_args()

In [3]:
file_names = 'non-overlapping_includingbidirectional_all_genes_newannotation'
promoterpref='promoters_5UTR'
Czechowski_gene_categories = f'../../data/output/{file_names}/genes/promoters_5UTR_czechowski_constitutive_variable_random.txt'
promoter_bed_file = f'../../data/output/{file_names}/FIMO/promoters_5UTR.bed'
output_genecat_prefix = 'Czechowski'

In [4]:
#make directory for the plots to be exported to
dirName = f'../../data/output/{file_names}/TFBS_enrichment'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/TFBS_enrichment  already exists


In [5]:
#make directory for the plots to be exported to
dirName = f'../../data/output/{file_names}/TFBS_enrichment/plots'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/TFBS_enrichment/plots  already exists


In [6]:
#make directory for the plots to be exported to
dirName = f'../../data/output/{file_names}/TFBS_enrichment/gat_analysis'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/TFBS_enrichment/gat_analysis  already exists


In [7]:
#make directory for the plots to be exported to
dirName = f'../../data/output/{file_names}/TFBS_enrichment/other_analysis'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/TFBS_enrichment/other_analysis  already exists


In [8]:
def sort_files(promoter_bed_file,mapped_motifs_file,gene_categories):
    """sort the promoters, motifs and genes types data ready for gat TATA enrichment"""
    promoters = pd.read_table(promoter_bed_file, sep='\t', header=None)
    col = ['chr', 'start','stop','promoter_AGI','dot1','strand','source','type','dot2','attributes']
    promoters.columns = col
    #read in gene categories
    gene_cats = pd.read_table(gene_categories, sep='\t', header=None)
    cols = ['promoter_AGI','gene_type']
    gene_cats.columns = cols
    #merge with promoters
    promoters = pd.merge(gene_cats,promoters, on='promoter_AGI', how='left')
    #read in mapped_motifs file

    #select the correct columns depending on the number of columns in the file
    if len(motifs.columns) == 24:
        cols = ['chr', 'start', 'stop', 'promoter_AGI','dot1','strand','source','type','dot2','attributes',
            'motif_chr','motif_start','motif_stop','name_rep', 'score', 'motif_strand',
            'promoter_AGI2', 'p-value', 'q-value', 'matched_sequence', 'TF_name', 'TF_family', 'TF_AGI','bp_overlap']
        motifs.columns = cols
        #filter columns
        motifs = motifs[['motif_chr','motif_start','motif_stop','name_rep', 'score', 'motif_strand',
             'promoter_AGI2', 'p-value', 'q-value', 'matched_sequence', 'TF_name', 'TF_family', 'TF_AGI']]
        #rename columns
        cols = ['chr', 'start', 'stop', 'name_rep', 'score', 'strand', 'promoter_AGI', 'p-value', 'q-value', 'matched_sequence', 'TF_name', 'TF_family', 'TF_AGI']
        motifs.columns = cols
        
    elif len(motifs.columns) == 13:
        cols = ['chr', 'start', 'stop', 'name_rep', 'score', 'strand', 'promoter_AGI', 'p-value', 'q-value', 'matched_sequence', 'TF_name', 'TF_family', 'TF_AGI']
        motifs.columns = cols
        
    elif len(motifs.columns) == 17:
        cols = ['chr', 'start', 'stop', 'name_rep', 'score', 'strand', 'promoter_AGI', 'p-value', 
             'q-value', 'matched_sequence', 'TF_name','TF_family','TF_AGI','chr_openchrom','start_openchrom','stop_openchrom','bp_overlap' ]
        motifs.columns = cols
    
    #swap columns around so TF family is in column 4
    motifs = motifs[['chr', 'start', 'stop', 'TF_family', 'score', 'strand', 'promoter_AGI', 'p-value', 
             'q-value', 'matched_sequence', 'TF_name','name_rep','TF_AGI']]
    #write out the file
    bed = BedTool.from_dataframe(motifs).saveas(mapped_motifs_out)
    
    return promoters

In [7]:
def prepare_gat(df):
    """prepare files for running gat analysis - outputs a workspace file containing all promoters, a variable promoter file and a constitutive promoter file"""
    #make buffer to save promoters
    buffer = io.StringIO()
    df.to_csv(buffer,sep='\t', header=None, index = False)
    buffer.seek(0)
    #select only constitutive and variable genes
    #df = df[(df.gene_type == 'constitutive') | (df.gene_type == 'variable')]
    #reorder columns
    #with gene_type in fourth column for gat enrichment analysis
    df_reordered_gene_type = df[['chr','start','stop','gene_type', 'strand', 'source', 'attributes','promoter_AGI']]
    df_reordered_AGI = df[['chr','start','stop','promoter_AGI', 'strand', 'source', 'attributes','gene_type']]
    #with promoter_AGI in fourth column for other enrichment analysis such as CiiiDER
    #sort by chromosome and start
    sorted_motifs_gene_type = df_reordered_gene_type.sort_values(['chr','start'])
    sorted_motifs_AGI = df_reordered_AGI.sort_values(['chr','start'])
    #save bed file with all gene types
    bed = BedTool.from_dataframe(sorted_motifs_gene_type).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/{args.output_genecat_prefix}_{args.promoterpref}_allgenetypes_gat.bed')
    bed = BedTool.from_dataframe(sorted_motifs_gene_type).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/{args.output_genecat_prefix}_{args.promoterpref}_allgenetypes_AGI.bed')
    
    #save bed file with only constitutive and variable genetypes
    sorted_motifs_gene_type2 = sorted_motifs_gene_type[(sorted_motifs_gene_type.gene_type == 'constitutive') | (sorted_motifs_gene_type.gene_type == 'variable')]
    sorted_motifs_AGI2 = sorted_motifs_AGI[(sorted_motifs_AGI.gene_type == 'constitutive') | (sorted_motifs_AGI.gene_type == 'variable')]
    bed = BedTool.from_dataframe(sorted_motifs_gene_type2).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/{args.output_genecat_prefix}_{args.promoterpref}_constitutivevariable_gat.bed')
    bed = BedTool.from_dataframe(sorted_motifs_AGI2).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/{args.output_genecat_prefix}_{args.promoterpref}_constitutivevariable_AGI.bed')
    #make a new gat workspace file with all promoters (first 3 columns)
    bed = BedTool.from_dataframe(sorted_motifs_gene_type[['chr','start','stop']]).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/gat_analysis/{args.output_genecat_prefix}_{args.promoterpref}_workspace.bed')
    #select only variable promoters
    variable_promoters_extended = sorted_motifs_gene_type[sorted_motifs_gene_type['gene_type'] == 'variable']
    sorted_variable = variable_promoters_extended.sort_values(['chr','start'])
    bed = BedTool.from_dataframe(sorted_variable).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/gat_analysis/{args.output_genecat_prefix}_{args.promoterpref}_variable_gat.bed')
    #same again for other enrichment analyses
    variable_promoters_extended_agi = sorted_motifs_AGI[sorted_motifs_AGI['gene_type'] == 'variable']
    sorted_variable_AGI = variable_promoters_extended_agi.sort_values(['chr','start'])
    bed = BedTool.from_dataframe(sorted_variable_AGI).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/other_analysis/{args.output_genecat_prefix}_{args.promoterpref}_variable_AGI.bed')
    #make a constitutive only file
    constitutive_promoters = sorted_motifs_gene_type[sorted_motifs_gene_type['gene_type'] == 'constitutive']
    sorted_constitutive = constitutive_promoters.sort_values(['chr','start'])
    bed = BedTool.from_dataframe(sorted_constitutive).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/gat_analysis/{args.output_genecat_prefix}_{args.promoterpref}_constitutive_gat.bed')
    #same again for other enrichment analyses with AGI in fourth column
    constitutive_promoters_AGI = sorted_motifs_AGI[sorted_motifs_AGI['gene_type'] == 'constitutive']
    sorted_constitutive_AGI = constitutive_promoters_AGI.sort_values(['chr','start'])
    bed = BedTool.from_dataframe(sorted_constitutive_AGI).saveas(f'../../data/output/{args.file_names}/TFBS_enrichment/other_analysis/{args.output_genecat_prefix}_{args.promoterpref}_constitutive_AGI.bed')


In [8]:
promoters = sort_files(promoter_bed_file,Czechowski_gene_categories)

In [9]:
#create gat files
prepare_gat(promoters)

,chr,start,stop,gene_type,strand,source,attributes,promoter_AGI
186,1,344515,346051,variable,+,araport11,ID=gene:AT1G02000;Name=GAE2;biotype=protein_co...,AT1G02000
117,1,792838,793314,variable,-,araport11,ID=gene:AT1G03240;biotype=protein_coding;descr...,AT1G03240
143,1,864019,865372,variable,-,araport11,ID=gene:AT1G03457;biotype=protein_coding;descr...,AT1G03457
187,1,1000629,1001472,variable,+,araport11,ID=gene:AT1G03920;biotype=protein_coding;descr...,AT1G03920
193,1,1315823,1316918,variable,+,araport11,ID=gene:AT1G04700;biotype=protein_coding;descr...,AT1G04700
...,...,...,...,...,...,...,...,...
158,5,22956628,22957985,variable,+,araport11,ID=gene:AT5G56750;Name=NDL1;biotype=protein_co...,AT5G56750
80,5,23792382,23793238,constitutive,+,araport11,ID=gene:AT5G58920;biotype=protein_coding;descr...,AT5G58920
120,5,24110303,24111442,variable,+,araport11,ID=gene:AT5G59845;Name=GASA10;biotype=protein_...,AT5G59845
182,5,24326665,24327827,variable,-,araport11,ID=gene:AT5G60490;Name=FLA12;biotype=protein_c...,AT5G60490
